### Downloading of the required libraries

In [1]:
pip install --upgrade torch

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install --upgrade transformers


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade torchvision

Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip install faiss-cpu

### Importing the required Libraries

In [5]:
import transformers
from transformers import DPRContextEncoderTokenizer,DPRContextEncoder,DPRQuestionEncoderTokenizer,DPRQuestionEncoder

C:\Users\rajve\miniconda3\envs\nlp_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
import numpy as np
import torch
from transformers import AutoModelForCausalLM,AutoTokenizer
import pandas as pd

### Loading The Data (The Company Policies)

In [7]:
with open('companyPolicies.txt','r') as f:
    text_input=f.read()

In [8]:
text_input

"1.\tCode of Conduct\n\nOur Code of Conduct outlines the fundamental principles and ethical standards that guide every member of our organization. We are committed to maintaining a workplace that is built on integrity, respect, and accountability.\nIntegrity: We hold ourselves to the highest ethical standards. This means acting honestly and transparently in all our interactions, whether with colleagues, clients, or the broader community. We respect and protect sensitive information, and we avoid conflicts of interest.\nRespect: We embrace diversity and value each individual's contributions. Discrimination, harassment, or any form of disrespectful behavior is unacceptable. We create an inclusive environment where differences are celebrated and everyone is treated with dignity and courtesy.\nAccountability: We take responsibility for our actions and decisions. We follow all relevant laws and regulations, and we strive to continuously improve our practices. We report any potential violati

In [9]:
def split_para(text):
    text=[p.strip() for p in text.split("\n")]
    return text

In [10]:
text_input=split_para(text_input)
paragraphs=text_input

In [11]:
text_input

['1.\tCode of Conduct',
 '',
 'Our Code of Conduct outlines the fundamental principles and ethical standards that guide every member of our organization. We are committed to maintaining a workplace that is built on integrity, respect, and accountability.',
 'Integrity: We hold ourselves to the highest ethical standards. This means acting honestly and transparently in all our interactions, whether with colleagues, clients, or the broader community. We respect and protect sensitive information, and we avoid conflicts of interest.',
 "Respect: We embrace diversity and value each individual's contributions. Discrimination, harassment, or any form of disrespectful behavior is unacceptable. We create an inclusive environment where differences are celebrated and everyone is treated with dignity and courtesy.",
 'Accountability: We take responsibility for our actions and decisions. We follow all relevant laws and regulations, and we strive to continuously improve our practices. We report any p

### Loading The DPR Context Encoder

In [12]:
context_tokenizer=DPRContextEncoderTokenizer.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')
context_tokenizer

DPRContextEncoderTokenizer(name_or_path='facebook/dpr-ctx_encoder-single-nq-base', vocab_size=30522, model_max_length=1000000000000000019884624838656, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [13]:
text=[("How are you","I am Fine"),("Code of Conduct","Our Code of Conduct outlines the fundamental principles and ethical standards that guide every member of our organization.")]
tokenized_text=context_tokenizer(text,return_tensors='pt',padding=True,turnucation=True,max_length=256)
tokenized_text

C:\Users\rajve\miniconda3\envs\nlp_env\lib\site-packages\transformers\tokenization_utils_base.py:2299: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


{'input_ids': tensor([[  101,   100,  2024,  2017,   102,   100,  2572,   100,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0],
        [  101,   100,  1997,   100,   102,   100,   100,  1997,   100, 22106,
          1996,  8050,  6481,  1998, 12962,  4781,  2008,  5009,  2296,  2266,
          1997,  2256,  3029,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0],
        [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]])}

In [14]:
context_encoder=DPRContextEncoder.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')

Loading weights: 100%|█| 197/197 [00:00<00:00, 289.62it/s, Materializing param=ctx_encoder.bert_model.encoder.layer.11.
DPRContextEncoder LOAD REPORT from: facebook/dpr-ctx_encoder-single-nq-base
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
ctx_encoder.bert_model.pooler.dense.weight | UNEXPECTED |  | 
ctx_encoder.bert_model.pooler.dense.bias   | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [15]:
encoded_text=context_encoder(**tokenized_text)

In [16]:
encoded_text

DPRContextEncoderOutput(pooler_output=tensor([[-0.1174,  0.6313, -0.5366,  ..., -0.4621,  0.5005,  0.0636],
        [ 0.4567,  0.8397, -0.3309,  ..., -0.4183,  0.3213,  0.1116]],
       grad_fn=<SelectBackward0>), hidden_states=None, attentions=None)

### Function to perform this encoding on Given Company Policies

In [17]:
def encode_context(text_input,context_tokenizer,context_encoder):
    tokenized_text=context_tokenizer(text_input,return_tensors='pt',padding=True,turnucation=True,max_length=256)
    encoded_text=context_encoder(**tokenized_text)
    return encoded_text

In [18]:
text_input=encode_context(text_input,context_tokenizer,context_encoder)

In [19]:
text_input

DPRContextEncoderOutput(pooler_output=tensor([[ 0.1921,  0.4063, -0.5398,  ..., -0.1969,  0.5815,  0.1131],
        [-0.6405, -0.4024, -0.4379,  ..., -0.2095,  0.1768,  0.1428],
        [ 0.6829,  0.8119, -0.0349,  ..., -0.3903,  0.0764, -0.1171],
        ...,
        [ 0.0483,  0.2375,  0.1606,  ..., -0.0729, -0.2378, -0.2347],
        [ 0.0065,  0.3086,  0.1141,  ..., -0.5427, -0.1104, -0.2087],
        [-0.6405, -0.4024, -0.4379,  ..., -0.2095,  0.1768,  0.1428]],
       grad_fn=<SelectBackward0>), hidden_states=None, attentions=None)

In [20]:
emb=text_input.pooler_output
emb.shape

torch.Size([94, 768])

### Making FAISS Instance

In [21]:
import faiss
embedding_dim=text_input.pooler_output.shape[1]   
embedding_dim

768

In [22]:
index=faiss.IndexFlatL2(embedding_dim)

In [23]:
context_embeddings_np=np.array(text_input.pooler_output.detach()).astype('float32')

C:\Users\rajve\AppData\Local\Temp\ipykernel_7900\3936354669.py:1: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  context_embeddings_np=np.array(text_input.pooler_output.detach()).astype('float32')


In [24]:
index.add(context_embeddings_np)

### Example Query and response using Faiss

In [25]:
querry="Code of Conduct"

In [26]:
question_tokenizer=DPRQuestionEncoderTokenizer.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')
question_encoder=question_encoder=DPRQuestionEncoder.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')

Loading weights: 0it [00:00, ?it/s]
DPRQuestionEncoder LOAD REPORT from: facebook/dpr-ctx_encoder-single-nq-base
Key                                                                                  | Status     | 
-------------------------------------------------------------------------------------+------------+-
ctx_encoder.bert_model.encoder.layer.{0...11}.intermediate.dense.bias                | UNEXPECTED | 
ctx_encoder.bert_model.encoder.layer.{0...11}.output.dense.weight                    | UNEXPECTED | 
ctx_encoder.bert_model.encoder.layer.{0...11}.attention.self.query.bias              | UNEXPECTED | 
ctx_encoder.bert_model.encoder.layer.{0...11}.attention.self.value.weight            | UNEXPECTED | 
ctx_encoder.bert_model.encoder.layer.{0...11}.output.dense.bias                      | UNEXPECTED | 
ctx_encoder.bert_model.encoder.layer.{0...11}.attention.self.key.weight              | UNEXPECTED | 
ctx_encoder.bert_model.encoder.layer.{0...11}.attention.output.LayerNorm.bias  

In [27]:
tokenized_querry=question_tokenizer(querry,padding=True,turnucation=True,return_tensors='pt',max_len=256)
encoded_querry=question_encoder(**tokenized_querry).pooler_output.detach().numpy().astype('float32')

In [28]:
encoded_querry

array([[ 1.85058892e-01, -1.19465157e-01, -6.60231233e-01,
        -5.43776155e-01, -4.97903317e-01,  5.88654160e-01,
        -3.01712096e-01, -1.32290602e+00, -1.32525414e-01,
         7.33433187e-01,  3.45826477e-01,  9.13979530e-01,
         2.43029535e-01,  2.30542704e-01, -1.73824668e-01,
        -4.54152137e-01, -6.21120393e-01,  1.66872728e+00,
         1.91798019e+00, -1.00815153e+00,  1.67567682e+00,
         1.18151140e+00, -3.42150927e-01, -3.56410205e-01,
         2.18557143e+00,  8.49229515e-01, -1.57574967e-01,
        -9.60024714e-01, -9.28405747e-02,  3.09502184e-01,
        -4.74506877e-02,  1.69007683e+00,  1.21260118e+00,
         2.15115044e-02,  1.27097787e-02, -1.18234348e+00,
        -5.75745583e-01,  1.36874688e+00, -1.40806580e+00,
        -6.14058673e-01, -1.29910260e-01,  1.16208434e+00,
        -1.67624578e-01,  1.39827609e+00,  7.24658549e-01,
         1.38887346e+00,  3.91046077e-01, -1.51802719e+00,
         1.07329452e+00, -1.30355358e+00, -3.59291196e-0

In [29]:
D,I=index.search(encoded_querry,k=5)

In [30]:
I

array([[77, 60, 69,  0, 66]])

In [32]:
for i in I[0]:
    print(paragraphs[i])

The Anti-Discrimination and Harassment Policy is a testament to the commitment of this organization in fostering a workplace that is free from discrimination, harassment, and any form of unlawful bias. This policy applies to every individual within the organization, including employees, contractors, visitors, and clients.
Policy Objective: The Drug and Alcohol Policy is established to establish clear expectations and guidelines for the responsible use of drugs and alcohol within the organization. This policy aims to maintain a safe, healthy, and productive workplace.
Your adherence to this policy is appreciated as it helps to maintain a safe and drug-free workplace for all.
1.	Code of Conduct
Treatment and Assistance: Employees with substance abuse issues are encouraged to seek help. The organization is committed to providing support, resources, and information to assist those seeking treatment.
